# NER and Sentiment

In this section we will work through applying basic sentiment analysis to our data using a pre-built *distilBERT* model from the **Flair** library. We will then use our organization labels captured through NER in the previous section to create a list of organizations with the highest and lowest average sentiment scores.

In [1]:
import pandas as pd
import flair

We initialize the English sentiment model `en-sentiment`:

In [2]:
model = flair.models.TextClassifier.load('en-sentiment')

2023-02-10 19:40:47,585 loading file /Users/druk/.flair/models/sentiment-en-mix-distillbert_4.pt


For each sample there are a few steps we need to take to create the sentiment score. We need to tokenize the input text, make a prediction, extract the direction (*positive* or *negative*) and confidence (a score from 0 to 1). If this is new to you, we cover the Flair sentiment model in more depth in **TK insert link**.

The following function carries out each of these steps for a single extract:

In [3]:
def get_sentiment(text):
    # tokenize input text
    sentence = flair.data.Sentence(text)
    # make sentiment prediction
    model.predict(sentence)
    # extract sentiment direction and confidence (label and score) object
    sentiment = sentence.labels[0]
    return sentiment

We now need to load our previously processed dataframe (which includes the *organizations* column) and `apply` the `get_sentiment` function to the *selftext* column. These sentiment scores will then be stored in a new *sentiment* column.

In [4]:
# load data
df = pd.read_csv('./data/processed/reddit_investing_ner.csv', sep='|')
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86,"['Fed', 'St. Louis Fed', 'Bostic', 'Treasury',..."
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83,['The Bear Cave](https://thebearcave.substack....
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53,"['OWN', 'ITM', 'REALLY', 'Collateral']"
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62,"['the NQ on Feb 22', 'the new micro futures', ..."
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67,[]


In [5]:
# get sentiment
df['sentiment'] = df['selftext'].apply(get_sentiment)
df.head()

,created_utc,downs,id,score,selftext,subreddit,title,ups,upvote_ratio,organizations,sentiment
0,1.614290e+09,0.0,t3_lshtjn,10.0,Bloomberg article: [https://www.bloomberg.com/...,investing,Fed Views Rising Yields as Bullish Sign Reflec...,10.0,0.86,"['Fed', 'St. Louis Fed', 'Bostic', 'Treasury',...","Sentence: ""Bloomberg article : [ https :// www..."
1,1.614286e+09,0.0,t3_lsgahw,56.0,Given the recent downturn in stocks especially...,investing,ARK ETFs implosion risk ------------------------,56.0,0.83,['The Bear Cave](https://thebearcave.substack....,"Sentence: ""Given the recent downturn in stocks..."
2,1.614283e+09,0.0,t3_lsf8td,1.0,[https://twitter.com/desogames/status/13649710...,investing,The Counter-Party Risk Bubble,1.0,0.53,"['OWN', 'ITM', 'REALLY', 'Collateral']","Sentence: ""[ https :// twitter.com / desogames..."
3,1.614282e+09,0.0,t3_lsf3nh,6.0,"When you think of futures, what comes to your ...",investing,Futures were made for days like these,6.0,0.62,"['the NQ on Feb 22', 'the new micro futures', ...","Sentence: ""When you think of futures , what co..."
4,1.614278e+09,0.0,t3_lsdcib,3.0,I've been on this sub for quite some time and ...,investing,Let's talk about liquidity premiums,3.0,0.67,[],"Sentence: ""I 've been on this sub for quite so..."


Now we need to extract each of the organizations alongside it's sentiment score. We will then loop through each, tallying up a total sentiment score and count.

Before we do that, we need to convert each value in the *organizations* column to a list (they are currently strings because we cannot save Python lists to file within Pandas dataframes, they are automatically converted to strings).

In [6]:
import ast

df['organizations'] = df['organizations'].apply(lambda x: ast.literal_eval(x))

In [7]:
# initialize sentiment dictionary
sentiment = {}

# loop through dataframe and extract org labels and sentiment scores into sentiment dictionary
for i, row in df.iterrows():
    # extract sentiment direction and score
    direction = row['sentiment'].value
    score = row['sentiment'].score
    # loop through each label in organizations column
    for org in row['organizations']:
        # check if org label exists in sentiment dictionary already
        if org not in sentiment.keys():
            # if it doesn't, initialize new entry in dictionary
            sentiment[org] = {'POSITIVE': [], 'NEGATIVE': []}
        # append positive/negative score to respective dictionary entry
        sentiment[org][direction].append(score)

In [8]:
sentiment['ARK']

{'POSITIVE': [0.99809330701828, 0.7470293641090393, 0.999733030796051],
 'NEGATIVE': [0.9974728226661682,
  0.6304867267608643,
  0.6143212914466858,
  0.994274914264679,
  0.9999703168869019,
  0.9982093572616577,
  0.9989243149757385,
  0.9981945157051086,
  0.9999686479568481]}

Now we can loop through each organization entry in the sentiment dictionary and calculate an average positive, and average negative score:

In [9]:
# initialize sentiment list
avg_sentiment = []

# loop through each organization
for org in sentiment.keys():
    # get number of positive and negative ratings
    freq = len(sentiment[org]['POSITIVE']) + len(sentiment[org]['NEGATIVE'])
    for direction in ['POSITIVE', 'NEGATIVE']:
        # assign to variable for cleaner code
        score = sentiment[org][direction]
        # if there are no entries, set to 0
        if len(score) == 0:
            sentiment[org][direction] = 0.0
        else:
            # otherwise calculate total
            sentiment[org][direction] = sum(score)
    # now calculate total amount
    total = sentiment[org]['POSITIVE'] - sentiment[org]['NEGATIVE']
    # and the average score
    avg = total/freq
    # add to sentiment list
    avg_sentiment.append({
        'entity': org,
        'positive': sentiment[org]['POSITIVE'],
        'negative': sentiment[org]['NEGATIVE'],
        'frequency': freq,
        'score': avg
    })

In [10]:
sentiment_df = pd.DataFrame(avg_sentiment)
sentiment_df.head()

,entity,positive,negative,frequency,score
0,Fed,2.326648,10.933105,14,-0.614747
1,St. Louis Fed,0.000000,0.991645,1,-0.991645
2,Bostic,0.000000,0.991645,1,-0.991645
3,Treasury,0.610765,3.990731,5,-0.675993
4,the Atlanta Fed’s,0.000000,0.991645,1,-0.991645


Immediately we can see we have a lot of entities which have appeared once in our dataset, and because of this their score will be pushed to one extreme or the other. We can filter out anything with less than or equal to a frequency of `3` to remove many of these instances:

In [11]:
sentiment_df = sentiment_df[sentiment_df['frequency'] > 3]
sentiment_df

,entity,positive,negative,frequency,score
0,Fed,2.326648,10.933105,14,-0.614747
3,Treasury,0.610765,3.990731,5,-0.675993
7,ARK,2.744856,8.231823,12,-0.457247
9,ITM,0.000000,3.790710,4,-0.947677
16,eBay,1.879811,2.979899,5,-0.220018
...,...,...,...,...,...
1520,Tilray,0.944136,4.664320,6,-0.620031
1553,PLTR,1.624318,1.911521,4,-0.071801
1624,LMND,0.000000,4.746245,5,-0.949249
2119,Sony,4.888053,1.970413,7,0.416806


Here we have some more relevant information. We can see a few items that we can remove through the `BLACKLIST` covered in earlier sections such as *Fed* and *Treasury*, but nonetheless this list is looking much better than before. We can apply `sort` to search for the entities with the highest overall score:

In [12]:
sentiment_df.sort_values('score', ascending=False).head(10)

,entity,positive,negative,frequency,score
1403,IBM,2.965104,0.883971,4,0.520283
345,TAM,6.161803,1.880312,9,0.475721
1495,Yahoo Finance,2.310291,0.624828,4,0.421366
2119,Sony,4.888053,1.970413,7,0.416806
504,Intel,4.226028,1.953940,7,0.324584
481,Company,7.915097,4.899027,14,0.215434
196,ER,2.690315,1.999680,5,0.138127
353,YouTube,2.936317,2.180855,6,0.125910
200,Value,1.968677,1.735053,4,0.058406
100,QQQ,2.730508,2.435801,6,0.049118


Very quickly we've got our results that we have pulled together using simple, ready-to-use models, and **zero** text preprocessing. With further fine-tuning, and process development, these already good results can become great. Which we will cover soon.